In [1]:
from pandas import pandas as pd
#install with pip
from pandas import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from urllib.error import HTTPError

import datetime
import re
import sqlite3
import urllib
import csv
import datetime

In [2]:
# Read in state summary report for District, filter for needed fields and assign headers
insp = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv", 
                   usecols=[2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 17, 18, 80, 81], encoding="ISO-8859-1")
insp.columns = ["county", "licnum", "sitename", "streetaddy", "cityaddy", "zip",
                "inspnum", "insptype", "inspdispos", "inspdate", "totalvio", "highvio", "licid", "visitid"]
# county_sought = 'Marion' # uncomment if particular county sought
# insp = insp[(insp.county == county_sought)] # uncomment if particular county sought
insp.sitename = insp.sitename.str.title() #titlecase sitename
insp.streetaddy = insp.streetaddy.str.title() #titlecase street adress
insp.cityaddy = insp.cityaddy.str.title() #titlecase city
insp = insp.applymap(lambda x: str(x).strip() if len(str(x).strip()) else None) #strip whitespace, replace empty vals
insp['visitid'] = insp['visitid'].apply(int) # so it can be filtered against df below

result = []
result_for_urls = result # for url list
result_for_tuple = result.append("NULL") # later into db, space for user-input datetime

# takes LicenseID and VisitID, passes it into the urls for detailed reports later
for index, rows in insp.iterrows():
    visitid = rows['visitid']
    licid = rows['licid']
    urls = "https://www.myfloridalicense.com/inspectionDetail.asp? \
        InspVisitID= %s &id= %s" % (visitid, licid)
    urls = urls.replace(' ', '')
    result.append(urls)
urlList = result
urlList.pop(0) # get rid of first "Null" from append above
print(urlList)

['https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6632858&id=6605529', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6653464&id=4070634', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6602877&id=4070634', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6577708&id=2129302', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6577709&id=2129307', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6618656&id=5768844', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6606469&id=4332605', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6671597&id=2129510', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6671738&id=2129510', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6626820&id=6251692', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6586953&id=2129568', 'https://www.myfloridalicense.c

In [4]:
urlList = ['https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6632858&id=6605529', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6653464&id=4070634', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6602877&id=4070634', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6577708&id=2129302']# a shorter try for a list
details_list = []
def make_details_list(url):
    inspurl = url
    visitid = inspurl[inspurl.rfind("VisitID=") + 1:8]
    
    html = urlopen(url)
    soup = bs(html.read(), 'lxml')
    details = soup.find_all('font', {'face': 'verdana'})[10:] # relevant text on pages in verdana
    result = []
        
    for detail in details:
        row = [
        details[0].get_text('business'),
        details[24].get_text('visitid'),
        details[25].get_text('intermed_num'),
        details[26].get_text('basic_num'),      
        ]    
        row.append(details_list)    
    result.append(details_list)
    return details_list    

for url in urlList:
    make_details_list(urlList)

AttributeError: 'list' object has no attribute 'rfind'